In [6]:
import os
import pandas as pd
from opensky_api import OpenSkyApi

api = OpenSkyApi()

if not os.path.exists("../data/flight_states.csv"):
    current_dir = os.path.dirname(os.path.abspath(__file__))
    flights_csv_path = os.path.join(current_dir, "../data/flight_states.csv")
else:
    flights_csv_path = "../data/flight_states.csv"

if os.path.exists(flights_csv_path):
    data_csv = pd.read_csv(flights_csv_path)
    f_sts_df = pd.DataFrame(data_csv)
else:
    raise FileNotFoundError(f"CSV file not found: {flights_csv_path}")

In [7]:
f_sts_df = f_sts_df[f_sts_df.category > 0]
f_sts_df = f_sts_df.dropna(how="any")

In [8]:
def get_flights():
    return f_sts_df

FlightTrack(dict_values(['a3687e', 'AJT8143 ', 1728934233.0, 1728960651.0, [[1728934233, 18.4733, -65.9612, 609, 351, False], [1728934240, 18.482, -65.9634, 914, 343, False], [1728934247, 18.4901, -65.9664, 914, 339, False], [1728934255, 18.4997, -65.9704, 1219, 337, False], [1728934262, 18.5083, -65.9742, 1219, 336, False], [1728934276, 18.5255, -65.9822, 1524, 336, False], [1728934293, 18.5462, -65.9911, 1828, 337, False], [1728934307, 18.5633, -65.9987, 2133, 336, False], [1728934329, 18.591, -66.0114, 2438, 336, False], [1728934336, 18.6, -66.0155, 2743, 336, False], [1728934352, 18.6206, -66.0251, 3048, 335, False], [1728934359, 18.6299, -66.0295, 3352, 335, False], [1728934367, 18.6411, -66.0356, 3352, 328, False], [1728934374, 18.6497, -66.0425, 3352, 319, False], [1728934382, 18.6584, -66.0524, 3352, 309, False], [1728934390, 18.6658, -66.0638, 3657, 300, False], [1728934397, 18.6713, -66.0756, 3657, 293, False], [1728934404, 18.6755, -66.0882, 3657, 285, False], [1728934412, 1